In [16]:
Corpus="./data/train/MultiTrain.Shuffled.csv"
Devset="./data/dev/MultiDev.csv"
Testset="./data/test/MultiTest.csv"

In [17]:
from keras.preprocessing import text, sequence
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import text_normalization as tn
import gensim
import numpy as np
import pandas as pd
import io

def LoadData(Corpus,ClassesDict,Arabic=False): ## loading file
    DF=pd.read_csv(Corpus,converters={'text': str})
    labels=[ClassesDict[x] for x in DF['label'].tolist()]
    sentences=DF['text'].tolist()
    if Arabic:
        
        sentences=[tn.NormForWord2Vec(line) for line in sentences]
    TrueLabels=labels
    labels=to_categorical(np.asarray(labels), num_classes=len(ClassesDict))
    return sentences,labels,TrueLabels


In [18]:
import pandas as pd
import numpy as np
import re
import os

import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer

!pip install Tashaphyne
import pyarabic.arabrepr
from tashaphyne.stemming import ArabicLightStemmer

import random
from sklearn.model_selection import train_test_split


In [19]:
df=pd.read_csv(Corpus)
df

,Unnamed: 0,label,text
0,0,MSA,بالإضافة لقيام معلمو الجيزة للذهاب إلي جريدة ا...
1,1,MSA,بعدين والله حرام تجي تلقى الي واقف عند الاشاره...
2,2,DIAL_LEV,لمسه اليد مرتين واضحة جدا والحكم
3,3,DIAL_LEV,بخصوص الهاتريك عمرها ما راح تصير
4,4,DIAL_GLF,الله يجبر كسرهم ويرجع و لدهم اليوم قبل بكرى ،
...,...,...,...
86536,86536,DIAL_LEV,وبورسعيد حكايتها
86537,86537,MSA,نعم الان بعد لبس الزي الرسمي السعودي يكون لنا ...
86538,86538,DIAL_GLF,اتوقع والله اعلم انه يقصد بقية الشعب المسافرين...
86539,86539,DIAL_EGY,هي بيتبدأ


In [20]:
df_dev = pd.read_csv(Devset)
df_test = pd.read_csv(Testset)
df_dev

,Unnamed: 0,label,text
0,0,DIAL_EGY,تصور كل ده كله . . تلاقيني باحبك أكتر .
1,1,DIAL_EGY,أكتر من روحي باحبك .
2,2,DIAL_EGY,وتصور حب المخلص لحبيبه اللي ولا بأيه يتغير .
3,3,DIAL_EGY,تصور كل ده كله . . تلاقيني باحبك أكتر .
4,4,DIAL_EGY,أكتر من روحي باحبك . . بحقيقي باحبك أكتر .
...,...,...,...
10815,10815,MSA,نسبة الفقر في الاردن حسب الدراسات 13
10816,10816,MSA,بالرغم من كثرة الأخطاء الطبية إلا أننا لم نسمع...
10817,10817,MSA,. . . حمى الله
10818,10818,MSA,امل من الله العلي القدير ان يشفي الشباب الاخوا...


In [21]:
from nltk.corpus import stopwords
print(stopwords.words('arabic'))
#stop_words = set(stopwords.words('english'))


['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [22]:
#yall nrmove 7roof el gr w elstop words... de kol elly l2thom
stop_words =['من',
 'في',
 'على',
 'و',
 'فى',
 'يا',
 'عن',
 'مع',
 'ان',
 'هو',
 'علي',
 'ما',
 'اللي',
 'كل',
 'بعد',
 'ده',
 'اليوم',
 'أن',
 'يوم',
 'انا',
 'إلى',
 'كان',
 'ايه',
 'اللى',
 'الى',
 'دي',
 'بين',
 'انت',
 'أنا',
 'حتى',
 'لما',
 'فيه',
 'هذا',
 'واحد',
 'احنا',
 'اي',
 'كده',
 'إن',
 'او',
 'أو',
 'عليه',
 'ف',
 'دى',
 'مين',
 'الي',
 'كانت',
 'أمام',
 'زي',
 'يكون',
 'خلال',
 'ع',
 'كنت',
 'هي',
 'فيها',
 'عند',
 'التي',
 'الذي',
 'قال',
 'هذه',
 'قد',
 'انه',
 'ريتويت',
 'بعض',
 'أول',
 'ايه',
 'الان',
 'أي',
 'منذ',
 'عليها',
 'له',
 'ال',
 'تم',
 'ب',
 'دة',
 'عليك',
 'اى',
 'كلها',
 'اللتى',
 'هى',
 'دا',
 'انك',
 'وهو',
 'ومن',
 'منك',
 'نحن',
 'زى',
 'أنت',
 'انهم',
 'معانا',
 'حتي',
 'وانا',
 'عنه',
 'إلي',
 'ونحن',
 'وانت',
 'منكم',
 'وان',
 'معاهم',
 'معايا',
 'وأنا',
 'عنها',
 'إنه',
 'اني',
 'معك',
 'اننا',
 'فيهم',
 'د',
 'انتا',
 'عنك',
 'وهى',
 'معا',
 'آن',
 'انتي',
 'وأنت',
 'وإن',
 'ومع',
 'وعن',
 'معاكم',
 'معاكو',
 'معاها',
 'وعليه',
 'وانتم',
 'وانتي',
 '¿',
 '|']


In [23]:
def normalize(sentence):
    '''
    Argument:
        string of words
    return:
        string of words but standardize the words
    '''
    sentence = re.sub("[إأآا]", "ا", sentence)
    sentence = re.sub("ى", "ي", sentence)
    sentence = re.sub("ؤ", "ء", sentence)
    sentence = re.sub("ئ", "ء", sentence)
    sentence = re.sub("ة", "ه", sentence)
    sentence = re.sub("گ", "ك", sentence)
    return sentence


In [24]:
def removing_ar_stopwords(text):
    """
        Here we remove all Arabic stop words
        
    """
      # if read it from file
#     ar_stopwords_list = open("arabic_stopwords.txt", "r") 
#     stop_words = ar_stopwords_list.read().split("\n")
#     stop_words = []
    original_words = []
    words = word_tokenize(text) # it works on one hadith not list
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence


In [25]:
def clearReg(text):
    """
        This function for getting the normal values of out of lemmatization function
        that takse a string of dict as a 
        takes  : '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
        return : ['امر أخذ ما نهى انتهى']
    """
    each_lemma_word = []
    each_lemma_sentence = []
    for hadith in text:
        matches = re.findall(r'\"(.+?)\"',hadith)
        for word in matches:
            if len(word) >= 2 and word !='result':
                each_lemma_word.append(word)
        each_lemma_sentence.append(" ".join(each_lemma_word))
        each_lemma_word.clear()
    return each_lemma_sentence


In [26]:
def stemming_1(text):
    """
        This is first functoin for stemming and it's looks not good accurac, NLTK by ISRIStemmer.
    """
    st = ISRIStemmer()
    stemmend_words = []
    words = word_tokenize(text)
    for word in words:
        stemmend_words.append(st.stem(word))
    stemmed_sentence = " ".join(stemmend_words)
    return stemmed_sentence
        
    
    
def stemming_2(text):
    """
        This is Second functoin for stemming and it's looks good results, with built in library called Tashaphyne.
        The documentation here ==> https://pypi.org/project/Tashaphyne/
    
    """
    import pyarabic.arabrepr
    arepr = pyarabic.arabrepr.ArabicRepr()
    repr = arepr.repr

    from tashaphyne.stemming import ArabicLightStemmer
    ArListem = ArabicLightStemmer()

    Tweets_without_stop_words_and_with_normalization_and_with_stemming = []

    for tweet in Tweets_without_stop_words_and_with_normalization_and_with_stemming:
        words = word_tokenize(tweet)
        new_list = []
        for word in words:
            stem = ArListem.light_stem(word)
            stem = ArListem.get_stem()
            new_list.append(stem)

        tweets_sentence_with_stemming = " ".join(new_list)
        tweets_without_stop_words_and_with_normalization_and_with_stemming.append(hadith_sentence_with_stemming)
        
    return hadiths_without_stop_words_and_with_normalization_and_with_stemming


In [27]:
def lemmatization(text):
    """
        This function for lemma Arabic words by API, and it getting best result of the previous functions
        return a string dictinary like exactly '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
    """
    import http.client
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    hadith_dict = {}
    list_pyload_input = []
    list_pyload_out = []
    length = len(text)
    for h in text[:length]:
        q = '{"text":'+'"{}"'.format(h)+'}'
        list_pyload_input.append(q)
    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }
    for h in list_pyload_input:
        conn.request("POST", "/msa/webapi/lemma", h.encode('utf-8'), headers)
        res = conn.getresponse()
        data = res.read()
        list_pyload_out.append(data.decode("utf-8"))
        final_result = clearReg(list_pyload_out)     # call clearReg for clean the text
    return final_result


Upove some functions for like:

stemming_1 by ISRIStemmer from NLTK.

stemming_2 by Tashaphyne is an Arabic light stemmer(removing prefixes and suffixes) and give all possible segmentations.

lemmatization by Farasa API

By the experimental: lemmatization by Farasa have a good results.



In [29]:
df_Dupli =df[df['text'].duplicated()]
df_Dupli[108:118]

,Unnamed: 0,label,text
2128,2128,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2129,2129,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2130,2130,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2131,2131,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2132,2132,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2133,2133,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2134,2134,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2135,2135,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2136,2136,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...
2137,2137,MSA,ونسأل الله الحنّان المنّان ان يجعل ما نقوم به ...


In [30]:
df_Dupli_dev =df_dev[df_dev['text'].duplicated()]
df_Dupli_dev[108:118]

,Unnamed: 0,label,text
275,275,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
276,276,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
277,277,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
278,278,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
279,279,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
280,280,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
281,281,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
282,282,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
283,283,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .
284,284,DIAL_EGY,عايزين حكام م الصين . . . حكآمنا مرتشين . .


In [31]:
df_Dupli_test =df_test[df_test['text'].duplicated()]
df_Dupli_test[108:118]

,Unnamed: 0,label,text
3931,3931,DIAL_LEV,ومسكين اللي ما عنده وحدات
3944,3944,DIAL_LEV,اذاعة عبدوش الدايش . . . اذاعة الوحداتي
3952,3952,DIAL_LEV,مسكين اللي ما عنده
3953,3953,DIAL_LEV,وهذا الأردن أردنا . . . وابو حسين
3954,3954,DIAL_LEV,وهيك علمنا ابو حسين . . . شعب واحد مش
3957,3957,DIAL_LEV,اذاعة عبدوش الدايش . . . اذاعة الوحداتي
3963,3963,DIAL_LEV,مسكين اللي ما عنده
4009,4009,DIAL_LEV,مسكين اللي ما عنده وحدات
4041,4041,DIAL_LEV,مسكين اللي ما عنده
4042,4042,DIAL_LEV,مسكين اللي ما عنده وحدات


we have duplicated data so we will remove it 

In [32]:
print(df_dev['text'].duplicated().sum())
print(df_test['text'].duplicated().sum())

#df_dev['text'].drop_duplicates(inplace=True)
#df_test['text'].drop_duplicates(inplace=True)

1078
614


In [33]:
print(df_dev.isnull().sum())
df_test.isnull().sum()


Unnamed: 0      0
label           0
text          140
dtype: int64


Unnamed: 0      0
label           0
text          108
dtype: int64

In [34]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df_dev.dropna(inplace=True)
df_test.dropna(inplace=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85658 entries, 0 to 86540
Data columns (total 3 columns):
Unnamed: 0    85658 non-null int64
label         85658 non-null object
text          85658 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ MB


In [36]:
#df_dev.columns


Index(['Unnamed: 0', 'label', 'text'], dtype='object')

In [40]:
#df.drop(axis=1 , columns['Unnamed: 0'])

In [40]:
all_tweets_1 = []
all_labels_1 = []
all_removedIndex_1 = []
counter = 0

#hna n2s mn 86541 l 85658 3l4an kda h7zn index bta3 elee etms7
#bnsave el removed index l2sbab b7siaa
for tweet in df['text']:
    counter = counter +1;
    if(isinstance(tweet, str)):
        all_tweets_1.append(tweet)
        all_labels_1.append(df['label'].iloc[counter-1])
    else:
        all_removedIndex_1.append(counter-1)

    


In [41]:
all_labels_1[0:10]


['MSA',
 'MSA',
 'DIAL_LEV',
 'DIAL_LEV',
 'DIAL_GLF',
 'MSA',
 'MSA',
 'MSA',
 'MSA',
 'MSA']

In [42]:
df['label'].iloc[85652:]

86535         MSA
86536    DIAL_LEV
86537         MSA
86538    DIAL_GLF
86539    DIAL_EGY
86540    DIAL_LEV
Name: label, dtype: object

In [43]:
len(all_tweets_1)


85658

In [94]:
#print(all_tweets_1[:10])
#print(all_labels_1[:10])


In [87]:
df['text'].isnull().sum() 

0

yalaaa n3ml el pre processing 3l data

it take ~= 2 hours 

In [55]:
%%time

# preprocess the tweets
cleared_tweet_1 = []           # Removing stopwords
cleared_tweet_1_2 = []         # Normalization
cleared_tweet_1_2_3 = []       # Lematization

for tweet in all_tweets_1[0:85658]: #85658
    cleared_tweet_1.append(removing_ar_stopwords(tweet))         # Removing stopwords
for tweets in cleared_tweet_1:
    cleared_tweet_1_2.append(normalize(tweets))                   # Normalization
cleared_tweet_1_2_3 = lemmatization(cleared_tweet_1_2)           # Lematization
#By the experimental: lemmatization by Farasa have a good results.

print('The size of data:')
len(cleared_tweet_1), len(cleared_tweet_1_2), len(cleared_tweet_1_2_3)


The size of data:
Wall time: 28min 39s


(10680, 10680, 10680)

In [56]:
cleared_tweet_1_2_3

['ياراجل فن هو فن قال',
 'حاج تكسف مش كفاي روب كمان',
 'اللهم صبر ابتليتنا منظر زود قرف نشوف شارع',
 'ارحمونه فن خرب اقارب عالم',
 'ارحمونه فن خرب اقارب عالم',
 'اخت روب مش مكفي عامل جايب اخت مشرحه ناقص',
 'مجتش عنى اشمعنا هيف طلع اخت جت غلبانه لي ماتقوله اه قال',
 'نتفرج ولاد مصر شرم شيخ صيف',
 'ديما سمع ناس هرب ردو سمع ناس مات ردو سمع ناس مركب تتحبس تونس بيا اه سبب هل جاز غلي مركب مبقش أجاب مصريفه ناس قت طمع طره اي سبب 000000000 الله عون',
 'تخلي واضح استاذ كل عارف قرب حرب تكلم صراح لي عاجب وافق مش عاجب',
 'بعد تكلم عربي ظبط سياد ناقص حصاد مر عائش منطق تان اساتذ لا مش شايف مثل دولة سوريا غير بيعملوا',
 'رب ستر فتره جايه مقصل فساد نظام حالي سندال حرب نظام',
 'دخول موقع انسخ لينك فتح صفح',
 'اسف اتعودنا بلد حادث قطار قت',
 'امتي استراح بق حكومه',
 'رب شاء الله مفيش اصاب لا وفاة شكر موقع راءع سابع لان نش خبر فور وقوع yasser 1573 hotmail',
 'ادي نتيجه تجاهل كفاءة مءسس حكوميه خربوها ناس توع متصورين راح يساعدوهم تنفيذ توريث بلد اسف مش فاضل شيء يتورث بلد كان خرب',
 'علاء هي غير اعضاء لجنه ا

In [64]:
labels = pd.DataFrame(all_labels_1,columns=['label'])
labels.iloc[:]

,label
0,DIAL_EGY
1,DIAL_EGY
2,DIAL_EGY
3,DIAL_EGY
4,DIAL_EGY
...,...
10699,MSA
10700,MSA
10701,MSA
10702,MSA


In [89]:
#df_dev.isnull().sum()

Arabic_Tweets_Cleaned    0
new_labels               0
dtype: int64

In [ ]:
#here we combine the cleaned text with it's label after cleaning - normlize 

In [78]:
df_test = pd.DataFrame(cleared_tweet_1_2_3,columns=['Arabic_Tweets_Cleaned'])
df_test['new_labels'] = labels['label']
df_test[3300:3320]

,Arabic_Tweets_Cleaned,new_labels
3300,الله لايبلانا لا حول اقو,DIAL_GLF
3301,عشان قطع ارض زهق,DIAL_GLF
3302,حال,DIAL_GLF
3303,ارد نكلامك روع اتمني هاجم شعب اردني زار اردن ق...,DIAL_GLF
3304,قال زور اردن زار مر حلف ماعاد اعتب لاتعامل منف...,DIAL_GLF
3305,صار اسافر لبنان سوريا جو احسن اهان منفذ حدوديه...,DIAL_GLF
3306,الله مستعان عنى خال ضامن عاش ها دنياء,DIAL_GLF
3307,اول شي مهبول يانجم سهيل جريم شرف سمع قال امام ...,DIAL_GLF
3308,حمل ذكر غثي في عنطزه,DIAL_GLF
3309,قال ايمان,DIAL_LEV


In [ ]:
#save our data after cleaned

In [79]:
df_test.to_csv('Arabic_Data_test.csv', index=False)


In [44]:
df['text'].isnull().value_counts()

False    85658
Name: text, dtype: int64

In [132]:
df['label'].isnull().value_counts()

False    85658
Name: label, dtype: int64

In [ ]:
#df_all.drop_duplicated(inplace= True)

In [24]:
df_all.to_csv("Arabic_Data_cleaned_without_duplicated.csv")

In [30]:
df_all['labels_new'].value_counts()

MSA         43287
DIAL_GLF    15885
DIAL_EGY     9343
DIAL_LEV     8730
Name: labels_new, dtype: int64